In [2]:
import json
import pandas as pd
import numpy as np

In [3]:
route_data = {}
travel_times = {}
package_data = {}
actual_sequences = {}
invalid_sequences_scores = {}

with open("../data/model_build_inputs/route_data.json", "r") as f:
    route_data = json.load(f)

with open("../data/model_build_inputs/travel_times.json", "r") as f:
    travel_times = json.load(f)

with open("../data/model_build_inputs/package_data.json", "r") as f:
    package_data = json.load(f)

with open("../data/model_build_inputs/actual_sequences.json", "r") as f:
    actual_sequences = json.load(f)

with open("../data/model_build_inputs/invalid_sequence_scores.json", "r") as f:
    invalid_sequences_scores = json.load(f)

# Create a dataframe of the actual sequences
actual_sequences = pd.DataFrame(actual_sequences).T
actual_sequences = actual_sequences.reset_index(names ="route_id")

# Create a dataframe of the route data
route_data = pd.DataFrame(route_data).T
route_data = route_data.reset_index(names ="route_id")

# Create a dataframe of the travel times
travel_times = pd.DataFrame(travel_times).T
travel_times = travel_times.reset_index(names ="route_id")

# Create a dataframe of the package data
package_data = pd.DataFrame(package_data).T
package_data = package_data.reset_index(names ="route_id")

# Create a dataframe of the invalid sequence scores
invalid_sequences_scores  = pd.DataFrame(list(invalid_sequences_scores.items()), columns=['route_id', 'score'])


# route_data

In [6]:
route_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6112 entries, 0 to 6111
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   route_id               6112 non-null   object
 1   station_code           6112 non-null   object
 2   date_YYYY_MM_DD        6112 non-null   object
 3   departure_time_utc     6112 non-null   object
 4   executor_capacity_cm3  6112 non-null   object
 5   route_score            6112 non-null   object
 6   stops                  6112 non-null   object
dtypes: object(7)
memory usage: 334.4+ KB


In [4]:
route_data.head()

,route_id,station_code,date_YYYY_MM_DD,departure_time_utc,executor_capacity_cm3,route_score,stops
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,DLA3,2018-07-27,16:02:10,3313071.0,High,"{'AD': {'lat': 34.099611, 'lng': -118.283062, ..."
1,RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,DSE4,2018-07-28,15:44:41,4247527.0,High,"{'AC': {'lat': 47.689446, 'lng': -122.296071, ..."
2,RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,DSE5,2018-08-18,15:32:04,4247527.0,High,"{'AA': {'lat': 47.268001, 'lng': -122.5079, 't..."
3,RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,DLA9,2018-08-15,15:09:38,3313071.0,High,"{'AB': {'lat': 33.823076, 'lng': -118.058727, ..."
4,RouteID_0021a2aa-780f-460d-b09a-f301709e2523,DLA7,2018-08-05,14:23:26,3313071.0,High,"{'AA': {'lat': 33.84364, 'lng': -117.773651, '..."


In [9]:
route_data.stops[0]

{'AD': {'lat': 34.099611,
  'lng': -118.283062,
  'type': 'Dropoff',
  'zone_id': 'P-12.3C'},
 'AF': {'lat': 34.101587,
  'lng': -118.291125,
  'type': 'Dropoff',
  'zone_id': 'A-1.2D'},
 'AG': {'lat': 34.089727,
  'lng': -118.28553,
  'type': 'Dropoff',
  'zone_id': 'A-2.1A'},
 'BA': {'lat': 34.096132,
  'lng': -118.292869,
  'type': 'Dropoff',
  'zone_id': 'A-1.2C'},
 'BE': {'lat': 34.098482,
  'lng': -118.286243,
  'type': 'Dropoff',
  'zone_id': 'P-13.3B'},
 'BG': {'lat': 34.102251,
  'lng': -118.287403,
  'type': 'Dropoff',
  'zone_id': 'P-13.2A'},
 'BP': {'lat': 34.095585,
  'lng': -118.28179,
  'type': 'Dropoff',
  'zone_id': 'P-13.2C'},
 'BT': {'lat': 34.101474,
  'lng': -118.289609,
  'type': 'Dropoff',
  'zone_id': 'A-1.1D'},
 'BY': {'lat': 34.091721,
  'lng': -118.284539,
  'type': 'Dropoff',
  'zone_id': 'A-1.1A'},
 'BZ': {'lat': 34.098179,
  'lng': -118.281075,
  'type': 'Dropoff',
  'zone_id': 'P-13.1C'},
 'CA': {'lat': 34.100562,
  'lng': -118.290796,
  'type': 'Dropoff'

In [4]:
route_data.route_score.value_counts()

route_score
Medium    3292
High      2718
Low        102
Name: count, dtype: int64

# travel_times

In [7]:
travel_times.head()

,route_id,AD,AF,AG,BA,BE,BG,BP,BT,BY,...,GG,PH,EN,QN,SG,SX,SU,UK,VK,BN
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,"{'AD': 0.0, 'AF': 198.3, 'AG': 264.9, 'BA': 26...","{'AD': 209.8, 'AF': 0.0, 'AG': 348.3, 'BA': 22...","{'AD': 295.7, 'AF': 333.6, 'AG': 0.0, 'BA': 27...","{'AD': 284.0, 'AF': 219.5, 'AG': 288.2, 'BA': ...","{'AD': 107.9, 'AF': 178.9, 'AG': 308.2, 'BA': ...","{'AD': 136.3, 'AF': 90.6, 'AG': 264.2, 'BA': 2...","{'AD': 120.0, 'AF': 267.6, 'AG': 163.7, 'BA': ...","{'AD': 172.7, 'AF': 64.8, 'AG': 317.3, 'BA': 2...","{'AD': 251.3, 'AF': 341.9, 'AG': 82.8, 'BA': 2...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,NaN,NaN,"{'AC': 426.5, 'AE': 295.1, 'AG': 0.0, 'AN': 12...","{'AC': 321.3, 'AE': 140.3, 'AG': 330.5, 'AN': ...",NaN,NaN,"{'AC': 216.7, 'AE': 49.6, 'AG': 377.2, 'AN': 3...",NaN,"{'AC': 80.1, 'AE': 102.1, 'AG': 311.7, 'AN': 3...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,"{'AA': 244.7, 'AD': 0.0, 'AJ': 434.0, 'AP': 17...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,NaN,NaN,"{'AB': 290.8, 'AC': 69.8, 'AG': 0.0, 'AI': 310...",NaN,"{'AB': 230.4, 'AC': 265.4, 'AG': 319.1, 'AI': ...",NaN,NaN,NaN,"{'AB': 227.7, 'AC': 359.7, 'AG': 364.3, 'AI': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RouteID_0021a2aa-780f-460d-b09a-f301709e2523,NaN,"{'AA': 427.0, 'AB': 529.9, 'AF': 0.0, 'AG': 37...","{'AA': 241.8, 'AB': 334.3, 'AF': 359.1, 'AG': ...",NaN,NaN,"{'AA': 588.2, 'AB': 473.8, 'AF': 264.2, 'AG': ...","{'AA': 529.7, 'AB': 609.7, 'AF': 329.1, 'AG': ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
travel_times.AD[0]

{'AD': 0.0,
 'AF': 198.3,
 'AG': 264.9,
 'BA': 268.9,
 'BE': 89.4,
 'BG': 140.0,
 'BP': 138.4,
 'BT': 192.6,
 'BY': 216.6,
 'BZ': 71.5,
 'CA': 153.3,
 'CG': 298.9,
 'CK': 163.7,
 'CM': 219.6,
 'CO': 100.5,
 'CP': 276.5,
 'CW': 200.6,
 'DJ': 410.0,
 'DL': 175.9,
 'DN': 276.5,
 'DQ': 217.6,
 'EC': 21.8,
 'EH': 92.9,
 'EO': 123.8,
 'EX': 40.5,
 'EY': 110.7,
 'FF': 145.8,
 'FH': 182.9,
 'FY': 322.0,
 'GB': 227.1,
 'GN': 134.5,
 'GP': 263.5,
 'GS': 51.3,
 'GU': 200.8,
 'GW': 252.7,
 'HB': 233.7,
 'HG': 155.0,
 'HN': 272.5,
 'HO': 243.7,
 'HR': 102.8,
 'HT': 305.7,
 'HW': 213.2,
 'IA': 110.5,
 'IJ': 206.3,
 'IM': 142.5,
 'IP': 175.1,
 'IW': 163.5,
 'JH': 167.3,
 'JM': 288.5,
 'KA': 173.5,
 'KG': 123.2,
 'KJ': 202.4,
 'KM': 137.7,
 'KN': 160.0,
 'KP': 262.6,
 'KU': 124.1,
 'LB': 175.4,
 'LD': 208.7,
 'LG': 322.4,
 'LK': 139.9,
 'LY': 201.7,
 'MA': 132.4,
 'MO': 194.0,
 'MQ': 160.4,
 'MR': 231.4,
 'MW': 154.5,
 'NE': 265.8,
 'NL': 396.3,
 'NM': 295.4,
 'NR': 212.0,
 'NU': 53.6,
 'PB': 112.6,
 

# package_data

In [21]:
package_data.head()

,route_id,AD,AF,AG,BA,BE,BG,BP,BT,BY,...,GG,PH,EN,QN,SG,SX,SU,UK,VK,BN
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,{'PackageID_9d7fdd03-f2cf-4c6f-9128-028258fc09...,{'PackageID_15c6a204-ec5f-4ced-9c3d-472316cc77...,{'PackageID_3b28f781-242e-416e-9575-84c7188b82...,{'PackageID_a18e36e0-6b5a-45b7-bf7a-7c3a15bd5a...,{'PackageID_22c8f630-8ab9-40d5-a36d-38ef01761b...,{'PackageID_0934f8ee-81f4-4084-b54f-5fe3e6c64b...,{'PackageID_c734cbd5-768c-4c20-81df-f366c77627...,{'PackageID_a595f722-d466-4981-a14e-3c5d30cbd2...,{'PackageID_cb9b6e35-82f9-4d8e-a0a2-20c0407bf7...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,NaN,NaN,{'PackageID_62bd35ae-6fe9-40c0-8c58-0d1021a796...,{'PackageID_80d944eb-442a-422f-9d40-3ad4bb73d7...,NaN,NaN,{'PackageID_c5e97530-28a1-481f-83a5-4cdb0e4b53...,NaN,{'PackageID_aa53f115-f85d-416a-8e73-a07dc6f41e...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,{'PackageID_de2b05b8-2186-47b2-b0f1-640cd01b65...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,NaN,NaN,{'PackageID_1520728c-7a38-45a1-9e5d-123eb6d023...,NaN,{'PackageID_d3c08539-cea1-4079-a0a2-f064fc2083...,NaN,NaN,NaN,{'PackageID_edb2f6fa-d7b6-440b-a764-1d431fc366...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RouteID_0021a2aa-780f-460d-b09a-f301709e2523,NaN,{'PackageID_ef2e9ad8-43a7-4043-9144-faf6730da3...,{'PackageID_9f9e7677-9f0f-42c4-ac05-8015c1447d...,NaN,NaN,{'PackageID_0168363a-f9ab-4795-bfda-787073937e...,{'PackageID_6796d17b-a2f2-41ea-8dd3-8cfd2c2aac...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
package_data.AD[0]

{'PackageID_9d7fdd03-f2cf-4c6f-9128-028258fc09ea': {'scan_status': 'DELIVERED',
  'time_window': {'start_time_utc': nan, 'end_time_utc': nan},
  'planned_service_time_seconds': 59.3,
  'dimensions': {'depth_cm': 25.4, 'height_cm': 7.6, 'width_cm': 17.8}},
 'PackageID_5541e679-b7bd-4992-b288-e862f6c84ae7': {'scan_status': 'DELIVERED',
  'time_window': {'start_time_utc': '2018-07-27 16:00:00',
   'end_time_utc': '2018-07-28 00:00:00'},
  'planned_service_time_seconds': 59.3,
  'dimensions': {'depth_cm': 25.4, 'height_cm': 12.7, 'width_cm': 17.8}},
 'PackageID_84d0295b-1adb-4a33-a65e-f7d6247c7a07': {'scan_status': 'DELIVERED',
  'time_window': {'start_time_utc': nan, 'end_time_utc': nan},
  'planned_service_time_seconds': 59.3,
  'dimensions': {'depth_cm': 39.4, 'height_cm': 7.6, 'width_cm': 31.8}}}

# actual_sequences

In [22]:
actual_sequences.head()

,route_id,actual
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,"{'AD': 105, 'AF': 47, 'AG': 4, 'BA': 33, 'BE':..."
1,RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,"{'AC': 36, 'AE': 28, 'AG': 104, 'AN': 94, 'AS'..."
2,RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,"{'AA': 58, 'AD': 94, 'AJ': 41, 'AP': 124, 'AR'..."
3,RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,"{'AB': 122, 'AC': 39, 'AG': 27, 'AI': 16, 'AU'..."
4,RouteID_0021a2aa-780f-460d-b09a-f301709e2523,"{'AA': 43, 'AB': 54, 'AF': 72, 'AG': 31, 'AI':..."


In [11]:
actual_sequences.actual[0]

{'AD': 105,
 'AF': 47,
 'AG': 4,
 'BA': 33,
 'BE': 109,
 'BG': 53,
 'BP': 67,
 'BT': 49,
 'BY': 7,
 'BZ': 61,
 'CA': 43,
 'CG': 26,
 'CK': 86,
 'CM': 96,
 'CO': 54,
 'CP': 36,
 'CW': 8,
 'DJ': 28,
 'DL': 15,
 'DN': 37,
 'DQ': 35,
 'EC': 106,
 'EH': 101,
 'EO': 56,
 'EX': 104,
 'EY': 69,
 'FF': 88,
 'FH': 12,
 'FY': 76,
 'GB': 81,
 'GN': 42,
 'GP': 2,
 'GS': 64,
 'GU': 16,
 'GW': 74,
 'HB': 97,
 'HG': 87,
 'HN': 40,
 'HO': 80,
 'HR': 111,
 'HT': 3,
 'HW': 93,
 'IA': 57,
 'IJ': 20,
 'IM': 68,
 'IP': 94,
 'IW': 23,
 'JH': 10,
 'JM': 92,
 'KA': 48,
 'KG': 55,
 'KJ': 41,
 'KM': 51,
 'KN': 17,
 'KP': 22,
 'KU': 63,
 'LB': 11,
 'LD': 46,
 'LG': 27,
 'LK': 84,
 'LY': 83,
 'MA': 66,
 'MO': 89,
 'MQ': 70,
 'MR': 25,
 'MW': 50,
 'NE': 45,
 'NL': 29,
 'NM': 79,
 'NR': 18,
 'NU': 65,
 'PB': 110,
 'PJ': 73,
 'PS': 82,
 'PT': 14,
 'PX': 117,
 'QE': 85,
 'QM': 5,
 'QO': 107,
 'QX': 118,
 'RA': 71,
 'RG': 59,
 'RY': 75,
 'SC': 19,
 'SD': 115,
 'SF': 6,
 'SI': 95,
 'SQ': 99,
 'TC': 44,
 'TG': 1,
 'TH': 

# invalid_sequences_scores

In [18]:
invalid_sequences_scores.head()

,route_id,score
0,RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77,0.631275
1,RouteID_0016bc70-cb8d-48b0-aa55-8ee50bdcdb59,0.447396
2,RouteID_001948e9-4675-486d-9ec5-912fd8e0770f,0.585464
3,RouteID_001b4ee3-c4f2-467f-932b-c85524d1021f,0.696672
4,RouteID_0021a2aa-780f-460d-b09a-f301709e2523,1.246236


# Geospatial visualisation

In [33]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from matplotlib.colors import ListedColormap, to_hex
import matplotlib.pyplot as plt

# Set the route number
route_id = 'RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77'
route_number = route_data[route_data.route_id == route_id].index[0]

# Extract latitude and longitude into a list of tuples
lat_lng_list = []
for stop_data in route_data.stops[route_number].values():
    lat_lng_list.append((stop_data['lng'], stop_data['lat']))

# Extract zone sequence into a list
zone_list = []
for stop_data in route_data.stops[route_number].values():
    zone_list.append(stop_data['zone_id'])

# Create a GeoDataFrame
geometry = [Point(lng, lat) for lng, lat in lat_lng_list]
gdf = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")

# Add the values as a new column in the GeoDataFrame
gdf['zone_id'] = zone_list  # Ensure value_list matches the number of points

# Create a colormap for the unique values
unique_values = gdf['zone_id'].unique()
cmap = ListedColormap(plt.cm.tab10.colors[:len(unique_values)])  # Use a categorical colormap
color_mapping = {value: to_hex(cmap(idx)) for idx, value in enumerate(unique_values)}  # Convert to hex

# Map the colors to the GeoDataFrame
gdf['color'] = gdf['zone_id'].map(color_mapping)

# Display points with colors using the .explore() method
gdf.explore(color=gdf['color'], tooltip='zone_id', marker_kwds={'radius': 5, 'fill': True})

In [23]:
unique_values

array(['P-12.3C', 'A-1.2D', 'A-2.1A', 'A-1.2C', 'P-13.3B', 'P-13.2A',
       'P-13.2C', 'A-1.1D', 'A-1.1A', 'P-13.1C', 'A-1.1B', 'P-13.1D',
       'P-12.2D', 'A-1.2A', 'A-1.1C', 'P-12.3D', 'A-2.3B', 'A-2.3A',
       'A-2.2A', 'A-1.3A', 'P-13.3D', 'A-1.3C', 'P-12.1D', 'A-1.3B',
       'P-13.2D', 'P-13.1B', 'P-13.3A', 'A-1.3D', 'P-13.3C', nan,
       'P-13.2B'], dtype=object)

# TSP

In [18]:
# Set the route number
route_id = 'RouteID_00143bdd-0a6b-49ec-bb35-36593d303e77'
route_number = route_data[route_data.route_id == route_id].index[0]



In [19]:
zone_list

['P-12.3C',
 'A-1.2D',
 'A-2.1A',
 'A-1.2C',
 'P-13.3B',
 'P-13.2A',
 'P-13.2C',
 'A-1.1D',
 'A-1.1A',
 'P-13.1C',
 'A-1.2D',
 'A-1.1B',
 'P-13.1D',
 'P-12.2D',
 'P-13.2A',
 'A-1.2C',
 'A-1.2A',
 'A-1.1C',
 'A-1.2A',
 'A-1.2C',
 'A-1.2C',
 'P-12.3C',
 'P-12.3D',
 'P-13.2A',
 'P-12.3C',
 'P-13.2C',
 'P-13.1D',
 'A-1.2A',
 'A-2.3B',
 'A-2.3A',
 'A-1.2D',
 'A-2.2A',
 'P-13.1C',
 'A-1.3A',
 'P-13.3D',
 'P-12.2D',
 'P-13.1D',
 'A-1.3C',
 'A-2.3A',
 'P-13.3B',
 'A-2.2A',
 'P-12.1D',
 'P-13.2A',
 'A-1.3A',
 'P-13.2C',
 'P-12.2D',
 'A-1.3B',
 'A-1.2A',
 'P-12.1D',
 'A-1.1D',
 'P-13.2A',
 'A-1.2D',
 'P-13.2A',
 'A-1.3A',
 'A-1.3B',
 'P-13.1C',
 'A-1.2A',
 'A-1.2D',
 'A-1.1B',
 'P-13.1D',
 'P-13.1D',
 'P-13.2C',
 'P-13.1D',
 'P-13.2C',
 'A-1.1B',
 'A-1.1D',
 'A-1.2D',
 'A-1.1C',
 'A-2.3A',
 'A-1.3A',
 'P-13.1C',
 'P-13.3B',
 'P-13.3D',
 'P-13.2D',
 'A-1.3A',
 'P-13.1B',
 'P-13.1D',
 'A-2.1A',
 'P-12.3C',
 'P-13.3A',
 'P-13.2C',
 'P-13.1C',
 'A-2.3B',
 'A-1.3A',
 'P-13.1B',
 'A-1.1A',
 'P-12.2D',